# `pstree`

Implement a program that reformats the output of `ps` to display it as a tree.

Plan:

1. Parse the output of `ps` to get the process list.
2. Construct a tree based on the parent process id.
3. Traverse the tree depth-first to print out processes.


In [ ]:
!ps -axo ppid,pid,user,command

In [ ]:
from collections import namedtuple
import re
import subprocess
from typing import Sequence

In [ ]:
pattern = re.compile(r"(\d+)\s+(\d+)\s+(\w+)\s+(\/.+)")

Process = namedtuple("Process", ["ppid", "pid", "user", "command"])


def parse_ps_output() -> Sequence[Process]:
    output = subprocess.run(
        ["ps", "axo", "ppid,pid,user,command"], capture_output=True, check=True
    )
    return [
        Process._make(match.groups())
        for match in pattern.finditer(output.stdout.decode())
    ]


In [ ]:
output = parse_ps_output()
for i in range(10):
    print(output[i])

## Hacky tree

Constructing the tree is a little bit tricky. Given each line, how do we know if the current process's parent is already in the tree? I guess we can use another dictionary to track each seen pid and it's position in the tree.


In [ ]:
def construct_tree(output: Sequence[Process]) -> dict:
    tree = {}
    nodes = {}

    for process in output:
        if process.ppid not in nodes:
            tree[process] = {}
            nodes[process.pid] = tree[process]
            continue

        node = nodes[process.ppid]
        node[process] = {}
        nodes[process.pid] = node[process]

    return tree

In [ ]:
process_tree = construct_tree(output)

In [ ]:
def print_tree(tree: dict, depth: int = 0):
    if not tree:
        return

    for process in tree.keys():
        print(
            f"{'   ' * (depth - 1) if depth > 1 else ''}{'└─ ' if depth else ''}{process}"
        )
        print_tree(tree[process], depth + 1)


In [ ]:
print_tree(process_tree)

In [ ]:
def print_tree_iter(tree: dict):
    if not tree:
        return

    root = next(iter(tree.keys()))
    children = tree[root]
    stack = [(root, children, 0)]

    while stack:
        node, children, depth = stack.pop()
        print(
            f"{'   ' * (depth - 1) if depth > 1 else ''}{'└─ ' if depth else ''}{node}"
        )
        for process in reversed(children.keys()):
            stack.append((process, children[process], depth + 1))


In [ ]:
print_tree_iter(process_tree)

In [ ]:
def print_tree_bfs(tree: dict):
    if not tree:
        return

    root = next(iter(tree.keys()))
    children = tree[root]
    queue = [(root, children)]

    depth = 0

    while queue:
        size = len(queue)
        for _ in range(size):
            node, children = queue.pop(0)
            print(
                f"{'   ' * (depth - 1) if depth > 1 else ''}{'└─ ' if depth else ''}{node}"
            )
            for process in children.keys():
                queue.append((process, children[process]))
        depth += 1

In [ ]:
print_tree_bfs(process_tree)

## Notes

MacOS ships with the BSD version of `ps`, which does not have a `--forest` option (unlike the GNU version).


In [ ]:
!man ps